In [1]:
import sqlite3
import pandas as pd
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import numpy as np
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

import pickle

In [2]:
con = sqlite3.connect(r"C:\Users\reekithak\Downloads\drugs.db")

In [3]:
df = pd.read_sql("SELECT * from train_set4",con)

In [4]:
df

,cid,emb0,emb1,emb2,emb3,emb4,emb5,emb6,emb7,emb8,...,emb23,emb24,emb25,emb26,emb27,emb28,emb29,emb30,emb31,target
0,3081361,3.717626,0.639790,1.385805,-3.248077,1.800350,-8.567437,11.963310,-7.790736,-4.454160,...,-4.025013,10.429837,-7.283005,-1.466492,0.953156,-4.969905,-7.563207,-6.804394,-1.095591,"1D4H,1FLT,1Y6A,1EVT,1IVO,1Y6A,1FLT,1JU5,2XB7,2..."
1,5327236,-0.372198,0.345462,-0.447692,0.762168,-1.128296,1.083966,-0.227938,-0.992710,1.245753,...,0.574602,1.091617,1.533445,0.424946,0.171480,-0.086621,-0.119564,-0.742002,-0.368440,"1D4H,1D4K"
2,5327235,-0.376090,0.413805,-0.163864,0.052044,-0.731339,0.850873,-0.480098,-0.923235,1.109870,...,0.447901,0.508694,1.325473,0.441002,0.283106,0.102159,-0.084240,-0.801526,-0.105570,"1D4H,1D4K"
3,5327234,-0.301883,0.371007,-0.358294,0.850007,-1.020490,0.863070,0.038748,-1.042293,0.331696,...,0.405777,0.951995,1.287835,0.329399,0.195070,-0.966746,0.120898,-0.616233,-0.832002,1D4H
4,3009319,-0.295947,0.277502,-0.462155,0.739710,-0.747083,1.014934,-0.141394,-1.019647,1.031712,...,0.431880,0.979879,1.497206,0.579064,0.091082,-0.131126,-0.098484,-0.374896,-0.564743,"1D4H,1D4K"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476479,76329005,1.166424,1.527692,-0.350568,-0.489907,-2.186424,2.682783,-1.475247,1.757537,-1.920095,...,-1.236230,1.318571,0.563403,-1.321251,-1.340777,-0.070082,-3.130717,3.829623,0.933152,"1CKP,4EUT,4EUT"
476480,76310925,0.459024,0.013794,-0.284438,0.591399,-0.824992,0.910093,-0.404519,-0.263853,-0.237646,...,-0.399889,0.261290,0.118542,1.157133,0.014574,-0.447181,0.129888,0.118966,0.176441,4EUT
476481,76321827,0.423295,1.466167,-0.525490,-0.840207,-1.431787,2.129326,-1.223760,2.291100,-1.785831,...,-1.279186,0.181222,0.508213,-0.541504,-1.260289,-0.797997,-2.023132,3.378190,1.288571,"1CKP,4EUT,4EUT"
476482,76325463,0.062179,-0.025483,-0.155615,0.543278,-1.155827,0.669949,-0.882610,-0.525503,-0.286333,...,-0.145780,0.796687,-0.149875,0.938495,-0.078610,-0.962729,-0.449951,0.446196,0.015125,4EUT


In [5]:
cids = df["cid"].unique()
unique_pbdid = set(",".join(df["target"].values.flatten().tolist()).split(","))
pbdid_dict = {v:k for k, v in dict(enumerate(unique_pbdid)).items()}

In [6]:
X = []
y = []
print("Preparing Data")
for cid in tqdm(cids, total=len(cids), leave=False):
    indices = np.where(df["cid"] == cid)[0]
    xi = df.iloc[indices, 1:-1].values
    X.append(xi)
    yi = np.zeros(len(pbdid_dict))
    targets = ",".join(df.iloc[indices, -1].values.tolist()).split(",")
    targets = [pbdid_dict[_id] for _id in targets if _id != ""]
    yi[targets] = 1
    y.append(yi)

X = np.array(X).reshape(-1, 32)
scaler = StandardScaler()
scaler.fit(X)
y = np.array(y)

np.save('X_4.npy',X)
np.save('y_4.npy',y)

  0%|                             | 3/476484 [00:00<5:24:21, 24.48it/s]

Preparing Data


In [30]:
#x is saved

In [2]:
X = np.load(r'C:\Users\reekithak\train0\X0.npy') ; y = np.load(r"C:\Users\reekithak\train0\y0.npy")

In [7]:
X.shape, y.shape

((476484, 32), (476484, 2162))

In [8]:
x_t, x_v, y_t, y_v = train_test_split(X, y, test_size=0.01, random_state=42)

In [7]:
import winsound 
  
freq = 100
dur = 50
  
# loop iterates 5 times i.e, 5 beeps will be produced. 
for i in range(0, 5):     
    winsound.Beep(freq, dur)     
    freq+= 100
    dur+= 50

In [8]:
X.shape , y.shape

((509983, 32), (509983, 2237))

In [18]:
x_t.shape , y_t.shape

((494683, 32), (494683, 2237))

In [20]:
y_t[0].shape

(2237,)

In [9]:
def batches(l, n):
    
    for i in range(0, len(l), n):
        if i + n >= len(l):
            yield l[i:len(l)]
        else:
            yield l[i:i+n]

In [10]:
from sklearn.linear_model import SGDClassifier
import random
import pickle
import time
import tqdm
from sklearn.naive_bayes import MultinomialNB
clf2 = MultinomialNB()

In [13]:
shuffledRange = list(range(len(x_t)))

In [14]:
len(shuffledRange)

494683

In [32]:
n_iter = 1

In [31]:
t1 = time.time()
for n in range(n_iter):
    random.shuffle(shuffledRange)
#     x_t = x_t[shuffledRange, :]
#     y_t = y_t[shuffledRange, :]
    print("ITER ",n)
    count = 0
    print("Init_Batches = ",len(shuffledX))
    for batch in batches(shuffledRange, 1024):
        x_batch = x_t[batch, :]
        y_batch = y_t[batch, :]
        clf2.partial_fit(x_batch, y_batch)
        print('Batch ',count)
        count+=1
pickle.dump(clf2,open("final_model.sav",'wb'))
print(time.time()-t1)

ITER  0
Init_Batches =  494683


ValueError: y should be a 1d array, got an array of shape (1024, 2237) instead.

In [31]:
#Simple Evaluation


In [11]:
import tensorflow as tf

In [12]:
def batch_gen(X, y, bs):
    indices = list(range(X.shape[0]))
    random.shuffle(indices)
    while True:
        for i in range(0, len(indices) - (len(indices) % bs), bs):
            if i + bs > len(indices):
                batch_x = X[i:, :]
                batch_y = y[i:, :]
            else:
                batch_x = X[i:i+bs, :]
                batch_y = y[i:i+bs, :]
            yield (batch_x, batch_y)

In [13]:
# for x_, y_ in batch_gen(X, y, bs=1024):
#     yhat = model.predict(x_)
#     print(yhat.shape, y_.shape)
#     break

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_shape=(32,), activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(1024, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2048, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(y.shape[1], activation="sigmoid")
])

In [15]:
# logits_loss = tf.keras.losses.SparseCategoricalCrossentropy()
# model.compile(optimizer="adam", loss=logits_loss, metrics=["accuracy"])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [16]:
bs = 1024
EPOCHS = 50
steps_per_epoch = x_t.shape[0] // bs
val_steps = x_v.shape[0] // bs
train_gen = batch_gen(x_t, y_t, bs=bs)
val_gen = batch_gen(x_v, y_v, bs=bs)

In [17]:
model.fit_generator(train_gen, steps_per_epoch = steps_per_epoch, epochs=EPOCHS, verbose=1)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
460/460 [==============================] - 1036s 2s/step - loss: 0.0122 - accuracy: 0.1456
Epoch 2/50
460/460 [==============================] - 133s 288ms/step - loss: 0.0015 - accuracy: 0.6093
Epoch 3/50
460/460 [==============================] - 116s 253ms/step - loss: 9.3419e-04 - accuracy: 0.7127
Epoch 4/50
460/460 [==============================] - 116s 253ms/step - loss: 7.7160e-04 - accuracy: 0.7334
Epoch 5/50
460/460 [==============================] - 117s 254ms/step - loss: 6.8493e-04 - accuracy: 0.7430
Epoch 6/50
460/460 [==============================] - 116s 253ms/step - loss: 6.3030e-04 - accuracy: 0.7490
Epoch 7/50
460/460 [==============================] - 116s 253ms/step - loss: 5.9044e-04 - accuracy: 0.7526
Epoch 8/50
460/460 [==============================] - 116s 252ms/step - loss: 5.6048e-04 - accuracy: 0.7565
Epoch 9/50
460/460 [==============================] - 117s 254ms/step 

In [18]:
model.save("graph0-ep54.h5")

In [19]:
#the above should be ep52 !~ rename don't forget ! 

In [20]:
model.save('graph4.h5')